In [143]:
import re
import json

In [ ]:
class ParsingError(Exception):
    """Exception personnalisée pour les erreurs de parsing"""
    def __init__(self, message, position=None):
        self.message = message
        self.position = position
        super().__init__(message)

**Objectif** : 
Créer une exception spécifique pour les erreurs de parsing, qui inclut un message d'erreur et une position dans le code pour faciliter le débogage.

**Explications** :
Le constructeur prend un message et une position (facultative) pour indiquer où l'erreur s'est produite.
Cette exception sera utilisée plus tard pour capturer et signaler les erreurs lors de l'analyse du code.

In [ ]:
# Classe principale du parser
class MTddVParser:
    def __init__(self, code):
        """Initialisation du parser avec le code à analyser."""
        self.code = code
        self.tokens = self.tokenize(code)  # Découper le code en tokens
        self.current_token_index = 0 # Index du token actuel
        self.errors = [] # Liste pour stocker les erreurs
        self.boucle_stack = []  # Pile pour gérer les blocs 'boucle' imbriqués

    def tokenize(self, code):
        """Tokeniser le code d'entrée en découpant sur les espaces blancs et les caractères spéciaux."""
        # Utilisation d'une expression régulière pour extraire les mots et les symboles
        tokens = re.findall(r'\w+|\(|\)|\}|fin|#|%', code)
        return tokens

    def current_token(self):
        """Retourner le token actuel ou None si aucun token n'est disponible."""
        if self.current_token_index < len(self.tokens):
            return self.tokens[self.current_token_index]
        return None

    def advance(self):
        """Avancer à l'élément suivant dans la liste des tokens."""
        self.current_token_index += 1

    def parse(self):
        """Effectuer le parsing du code et construire la structure d'instructions."""
        structure = [] # Liste pour stocker les instructions validées
        while self.current_token():
            try:
                instruction = self.parse_instruction() # Appeler la méthode pour analyser une instruction
                if instruction:
                    structure.append(instruction)
            except ParsingError as e:
                # En cas d'erreur, ajouter l'erreur à la liste et essayer de passer au token suivant
                self.errors.append({"error": e.message, "position": self.current_token_index})
                self.advance()

        # Vérification des blocs 'boucle' non fermés
        if self.boucle_stack:
            self.errors.append({"error": "Unclosed 'boucle' block(s) detected.", "position": self.current_token_index})

        # Vérification de la présence du caractère '#' à la fin du programme
        if self.tokens and self.tokens[-1] != "#":
            self.errors.append({"error": "Missing '#' at the end of the program.", "position": len(self.tokens) - 1})

        return structure # Retourne la structure d'instructions validées

    def parse_instruction(self):
        """Analyser une instruction individuelle dans le code."""
        token = self.current_token() # Récupérer le token actuel

        # Basic Instructions
        if token in ["I", "P", "G", "D", "0", "1"]:
            self.advance()
            return {"type": "instruction", "value": token}  # Ajouter l'instruction à la structure

        # Traitement des commentaires (%)
        elif token == "%":
            self.advance()  # Ignorer les commentaires
            return {"type": "comment"}
        
        # Traitement de l'indicateur de fin (#)        
        elif token == "#":
            self.advance()
            # Vérifier si '#' est bien le dernier token
            if self.current_token():
                self.errors.append({"error": "'#' must be the last instruction in the program.", "position": self.current_token_index})
            return {"type": "end_marker", "value": "#"}

        # Traitement des conditionnels ('si')
        elif token == "si":
            self.advance()
            condition = self.parse_condition()
            if condition not in ["0", "1"]:
                self.errors.append({"error": "Invalid condition in 'si' statement.", "position": self.current_token_index})
            inner_structure = []
            while self.current_token() != "}":
                if self.current_token() in ["boucle", "si"]:
                    inner_structure.append(self.parse_instruction())
                elif self.current_token() is None:
                    self.errors.append({"error": "Missing closing '}' for 'si' block", "position": self.current_token_index})
                    break
                else:
                    inner_structure.append(self.parse_instruction())
            if self.current_token() == "}":
                self.advance()
            else:
                self.errors.append({"error": "Expected '}' to close 'si' block", "position": self.current_token_index})
            return {"type": "si", "condition": condition, "content": inner_structure}

        # Traitement des boucles ('boucle')
        elif token == "boucle":
            self.boucle_stack.append("boucle")
            self.advance()
            inner_structure = [] # Structure interne de la boucle
            while self.current_token() != "}":
                if self.current_token() == "fin":
                    if not self.boucle_stack:
                        self.errors.append({"error": "'fin' outside of any 'boucle'", "position": self.current_token_index})
                    inner_structure.append({"type": "fin"})
                    self.advance()
                elif self.current_token() in ["boucle", "si"]:
                    inner_structure.append(self.parse_instruction())
                elif self.current_token() is None:
                    self.errors.append({"error": "Missing closing '}' for 'boucle' block", "position": self.current_token_index})
                    break
                else:
                    inner_structure.append(self.parse_instruction())
            if self.current_token() == "}":
                self.boucle_stack.pop() # Retirer le bloc 'boucle' de la pile
                self.advance()
            else:
                self.errors.append({"error": "Expected '}' to close 'boucle' block", "position": self.current_token_index})
            return {"type": "boucle", "content": inner_structure}

        elif token == "fin":
            # Traitement de 'fin' (fin de boucle)
            if not self.boucle_stack:
                self.errors.append({"error": "'fin' outside of any 'boucle' block", "position": self.current_token_index})
            self.advance()
            return {"type": "fin"}

        else:
            # Si le token est inconnu ou invalide
            self.errors.append({"error": f"Unknown instruction '{token}'", "position": self.current_token_index})
            self.advance()
            return {"type": "error", "value": f"unknown instruction '{token}'"}

    def parse_condition(self):
        """Parse condition in 'si' statement."""
        if self.current_token() == "(":
            self.advance()
            condition = self.current_token()
            self.advance()
            if self.current_token() == ")":
                self.advance()
                return condition
            else:
                self.errors.append({"error": "Expected ')' after condition in 'si'", "position": self.current_token_index})
        return None

##### Constructeur __init__ :

**Objectif** : Initialiser l'instance du parser avec le code source à analyser.
**Détails** :
self.code : Le code source qui sera analysé.
self.tokens : Liste des tokens générée en découpant le code par la méthode tokenize.
self.current_token_index : Position actuelle dans la liste des tokens.
self.errors : Liste qui contiendra les erreurs rencontrées durant l'analyse.
self.boucle_stack : Pile utilisée pour suivre les blocs boucle imbriqués et s'assurer qu'ils sont correctement fermés.


#### Méthode tokenize :

**Objectif** : Récupérer le token actuel à analyser.
**Détails** :
Si l'index actuel est inférieur à la taille de la liste de tokens, retourne le token à cette position.
Sinon, retourne None pour indiquer la fin des tokens.


#### Méthode advance :

**Objectif** : Avancer à l'élément suivant dans la liste des tokens.
**Détails** :
Augmente simplement l'index de current_token_index pour passer au token suivant.


#### Méthode parse :

**Objectif** : Analyser tout le code et produire la structure d'instructions, tout en collectant les erreurs.
**Détails** :
structure : Liste pour stocker les instructions valides extraites du code.
Pour chaque token, parse_instruction() est appelée pour analyser les instructions.
Si une exception ParsingError est levée, l'erreur est ajoutée à la liste errors et l'analyse continue avec le token suivant.
Si des blocs boucle ne sont pas fermés, ou si le programme ne se termine pas par un #, des erreurs sont ajoutées.


#### Méthode parse_instruction :

**Objectif** : Analyser une instruction à partir du token actuel et renvoyer la structure de cette instruction.
**Détails** : Selon le token, plusieurs blocs de code différents sont activés pour analyser des instructions spécifiques comme :
Instructions de base (I, P, G, D, 0, 1) : Retourne une instruction de type "instruction".
Commentaires (%) : Ignore le commentaire et le marque comme tel.
Bloc conditionnel (si) : Analyse une instruction conditionnelle avec une condition entre parenthèses.
Bloc de boucle (boucle) : Gère les boucles imbriquées et s'assure qu'elles sont correctement fermées.
Fin de boucle (fin) : S'assure que fin apparaît bien dans un bloc boucle.

### Méthode parse_condition :

**Objectif** : Analyser la condition à l'intérieur d'un bloc si (si condition).
**Détails** :
Si la condition commence par ( et se termine par ), elle est extraite et renvoyée. Si la parenthèse fermante est manquante, une erreur est ajoutée.

In [ ]:
# Charger et analyser le code depuis un fichier .ts
file_path = "err_programme_mtddv.ts" 

with open(file_path, "r", encoding="utf-8") as file:
    code = file.read()

# Créer une instance du parser et analyser le code
parser = MTddVParser(code)
# Effectuer l'analyse
parsed_structure = parser.parse()

# Choisir le nom du fichier de sortie en fonction de la présence d'erreurs
if parser.errors:
    output_filename = "./results/parsed_structure_with_errors.json"
    # Afficher les erreurs à l'écran
    for error in parser.errors:
        print(f"Erreur détectée : {error['error']} à la position {error['position']}")
else:
    output_filename = "./results/parsed_structure_valid.json"

# Créer le contenu JSON
output = {
    "parsed_structure": parsed_structure,
    "errors": parser.errors
}

# Sauvegarder le contenu dans le fichier approprié
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(output, f, indent=4, ensure_ascii=False)

print(f"Parsed structure and detected errors have been saved to '{output_filename}'")

Erreur détectée : Invalid condition in 'si' statement. à la position 10
Erreur détectée : Missing closing '}' for 'si' block à la position 51
Erreur détectée : Expected '}' to close 'si' block à la position 51
Parsed structure and detected errors have been saved to './results/parsed_structure_with_errors.json'
